In [2]:
import sys, pathlib
sys.path.append(str(pathlib.Path(r"c:\ml-projects").resolve()))
import sys
import os

# Add dlx to path
sys.path.insert(0, r'c:\ml-projects\dlx')

# Force NumPy backend
import dlx.utils.backend as backend_module
import numpy as np
from dlx.nn.optim import AdamW
from dlx.nn.tensor import Tensor
backend_module.xp = np

# Now import
from training.model import Model

CUDA not available
CUDA not available


In [122]:
VOCAB_SIZE = 6
D_MODEL = 4
N_HEADS = 2
MAX_SEQ_LEN = 18
PAD_IDX = 0
DEPTH = 3
temperature = 1.0
top_k = None

model = Model(
    vocab_size=VOCAB_SIZE,
    d_model=D_MODEL,
    max_seq_len=MAX_SEQ_LEN,
    pad_idx=PAD_IDX,
    n_heads=N_HEADS,
    transformer_depth=DEPTH,
    checkpoint_interval_seconds=3600,
    train_dir="",
    validation_dir="",
    checkpoint_dir="",
    epochs=1,
    mini_batch_per_step=8
)
x = np.array([[4, 3, 0, 5]], dtype=np.int32)

In [123]:
model.forward(x)

Tensor(data=[[[ 0.5821211   0.14637749 -0.01239725  0.23310004 -0.95350814
   -0.58259827]
  [-2.0239506  -1.7800325  -0.17124878  0.27141005  2.1315334
    1.6554675 ]
  [-1.9380119  -1.6958257  -0.17536205  0.2407715   2.0849853
    1.5957426 ]
  [ 0.123274   -1.0422896  -0.9044097  -0.47731978  1.1892995
    0.44133627]]], shape=(1, 4, 6), dtype=float32)

In [124]:
idx = x.copy()
for _ in range(10):
    logits = model.forward(idx)
    logits = logits[:, -1, :]
    logits = logits / temperature
    logits_np = logits.data[0]
    if top_k is not None:
        top_k_idx = np.argpartition(logits_np, -top_k)[-top_k:]
        mask = np.full_like(logits_np, -float('inf'))
        mask[top_k_idx] = logits_np[top_k_idx]
        logits_np = mask
    
    # Sample max probability
    probs = np.exp(logits_np - np.max(logits_np))
    probs = probs / np.sum(probs)
    next_token = np.argmax(probs)
    
    # Append to sequence
    next_token_array = np.array([[next_token]], dtype=np.int32)
    idx = np.concatenate([idx, next_token_array], axis=1)
    print(idx)
    

    current_position += 1



[[4 3 0 5 4]]
[[4 3 0 5 4 1]]
[[4 3 0 5 4 1 1]]
[[4 3 0 5 4 1 1 1]]
[[4 3 0 5 4 1 1 1 1]]
[[4 3 0 5 4 1 1 1 1 0]]
[[4 3 0 5 4 1 1 1 1 0 5]]
[[4 3 0 5 4 1 1 1 1 0 5 5]]
[[4 3 0 5 4 1 1 1 1 0 5 5 4]]
[[4 3 0 5 4 1 1 1 1 0 5 5 4 4]]


In [125]:
idx = x.copy()


# kv_cache is of shape (trasnformer_depth, batch_size, max_seq_len, d_model)
kv_shape = (DEPTH, 1, MAX_SEQ_LEN, D_MODEL)
kv_cache = {
    "k": Tensor(np.zeros(kv_shape)),
    "v": Tensor(np.zeros(kv_shape))
}


for i in range(len(idx[0])):
    current_position = i
    # print(f"Print kv cache after {i} tokens (position {current_position}) (in the prompt part of idx)")
    # print(kv_cache)
    logits = model.forward(idx[:,:i+1], kv_cache, current_position)


for _ in range(10):
    # print(f"Print kv cache after {i} tokens (position {current_position}) (in the generated part of idx)")
    # print(kv_cache)
    logits = model.forward(idx, kv_cache, current_position)
    logits = logits[:, -1, :]
    logits = logits / temperature
    logits_np = logits.data[0]
    if top_k is not None:
        top_k_idx = np.argpartition(logits_np, -top_k)[-top_k:]
        mask = np.full_like(logits_np, -float('inf'))
        mask[top_k_idx] = logits_np[top_k_idx]
        logits_np = mask
    
    # Sample from distribution
    probs = np.exp(logits_np - np.max(logits_np))
    probs = probs / np.sum(probs)
    next_token = np.argmax(probs)
    
    # Append to sequence
    next_token_array = np.array([[next_token]], dtype=np.int32)
    idx = np.concatenate([idx, next_token_array], axis=1)
    print(idx)
    

    current_position += 1



[[4 3 0 5 4]]
[[4 3 0 5 4 1]]
[[4 3 0 5 4 1 1]]
[[4 3 0 5 4 1 1 1]]
[[4 3 0 5 4 1 1 1 1]]
[[4 3 0 5 4 1 1 1 1 0]]
[[4 3 0 5 4 1 1 1 1 0 5]]
[[4 3 0 5 4 1 1 1 1 0 5 5]]
[[4 3 0 5 4 1 1 1 1 0 5 5 4]]
[[4 3 0 5 4 1 1 1 1 0 5 5 4 4]]
